In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_kraken_orderbooks_from_file
from hydradx.model.processing import get_omnipool_data_from_file

from hydradx.model.amm.arbitrage_agent import get_arb_swaps_simple, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket
from pprint import pprint

import json


In [2]:
with open('config/arbconfig.txt', 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_asset_list = []
buffer = {}

get_live_markets = False

if get_live_markets:

    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[arb_cfg['tkn_ids']] = arb_cfg['buffer']
        if tkn_pair not in ob_objs:
            order_book_url = 'https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1]
            ob_objs[tkn_pair] = get_kraken_orderbook(tkn_pair, order_book_url, archive=True)
            for tkn in tkn_pair:
                if tkn not in order_book_asset_list:
                    order_book_asset_list.append(tkn)
                    
else:
    ob_objs = get_kraken_orderbooks_from_file("data/input/")
    for arb_cfg in arb_list:
        tkn_pair = arb_cfg['order_book']
        buffer[arb_cfg['tkn_ids']] = arb_cfg['buffer']
        for tkn in tkn_pair:
            if tkn not in order_book_asset_list:
                order_book_asset_list.append(tkn)

In [3]:


RPC="wss://hydradx-rpc.dwellir.com"
N = 24

if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
buffer2 = {(asset_map[k[0]], asset_map[k[1]]): v for k, v in buffer.items()}

pprint(tokens)

{'ASTR': {'LRNA': 9493.01865460789, 'liquidity': 4454901.648347969},
 'BNC': {'LRNA': 6570.77820580829, 'liquidity': 733206.6287840898},
 'CFG': {'LRNA': 9055.74947148805, 'liquidity': 615212.4674276046},
 'DAI': {'LRNA': 12903.802000314236, 'liquidity': 338055.2939474438},
 'DAI001': {'LRNA': 31094.02789908123, 'liquidity': 818813.1999696387},
 'DOT': {'LRNA': 73166.58146171033, 'liquidity': 409282.6249603317},
 'GLMR': {'LRNA': 5292.16456910496, 'liquidity': 671965.6893360685},
 'HDX': {'LRNA': 20510.16976255191, 'liquidity': 111968011.89165848},
 'INTR': {'LRNA': 8375.72827588616, 'liquidity': 12753466.593879394},
 'USDT': {'LRNA': 84547.44835053636, 'liquidity': 2242324.613962},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41455.63313835339, 'liquidity': 31.34997588},
 'WETH': {'LRNA': 21496.13808288496, 'liquidity': 308.21785254735175},
 'WETH001': {'LRNA': 71242.34401923479, 'liquidity': 1023.9116639347923},
 'ZTG': {'LRNA': 4574.889374797759,

In [4]:
cex_fee = 0.0016
# buffer = 0.0002

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0005
# asset_fee = 0.0025


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()
print(order_book_asset_list)


['HDX', 'USD', 'DOT', 'XETH', 'ZUSD', 'ETH', 'XXBT', 'XBT', 'ASTR', 'CFG', 'BNC', 'GLMR']


In [5]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('DOT', 'iBTC'): ('DOT', 'XBT'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'DAI'): ('XETH', 'ZUSD'),
 ('WETH', 'DAI001'): ('XETH', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUS

In [6]:
all_swaps = get_arb_swaps_simple(op_state, cex, order_book_map, buffer=buffer2)
print(all_swaps)

[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'HDX', 'price': 0.0048, 'amount': 291162.27278636687, 'min_buy': 1404.5731977154612}, 'cex': {'trade': 'buy', 'buy_asset': 'HDX', 'sell_asset': 'USD', 'price': 0.004802399999999999, 'amount': 291162.27278636687}}, {'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'HDX', 'price': 0.0048, 'amount': 0.7353574876797354, 'min_buy': 0.003536005198955536}, 'cex': {'trade': 'buy', 'buy_asset': 'HDX', 'sell_asset': 'USD', 'price': 0.004802399999999999, 'amount': 0.7353574876797354}}, {'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'DOT', 'price': 4.4323, 'amount': 188.892, 'min_buy': 890.2646849532724}, 'cex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'USD', 'price': 4.434516149999999, 'amount': 188.892}}, {'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'DOT', 'price': 4.433, 'amount': 2.0, 'min_buy': 9.417997633695602}, 'cex': {'trade': 'buy', 'buy_asset': 'DOT', 'sell_asset': 'USD', 'pr

In [7]:
state = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit = calculate_profit(init_agent, agent, asset_map)

print(profit)
            

{'HDX': 0.0, 'USD': 1021.5998339653015, 'BTC': 1.3947486877441406e-05, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0.0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0.0, 'INTR': 0}
